In [1]:
%load_ext autoreload
%autoreload 2
from lib.feature_extractor import NASNetLargeExtractor

In [2]:
# download google nasnet large pre-trained model
model = NASNetLargeExtractor(32, 10)

In [3]:
# load cifar datasets
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

def preprocess_data(data_set):
    data_set /= 255.0
    return data_set

x_train = preprocess_data(x_train)
x_test = preprocess_data(x_test)

Y_train = to_categorical(y_train, num_classes = 10)
Y_test = to_categorical(y_test, num_classes = 10)

In [7]:
# extract features
features_train = model.extract(x_train[:50])
print(features_train.shape)

1/1 [==============================] - 0s 1ms/step
(50, 4032)


In [5]:
# save features
model.save_features("cifar10_train.csv")

In [19]:
# load features
model.load_features("cifar10_train.csv")

In [18]:
# use dense layer to test feature quality
print(model.features.shape)
history = model.test_feature_quality(Y_train[:50, :], epochs=2, batch_size=50, validation_data=(x_test[:18], Y_test[:18,:]))

(50, 4032)
1/1 [==============================] - 0s 922us/step
Epoch 1/2
1/1 [==============================] - 0s 152ms/step - loss: 28.0387 - accuracy: 0.5200 - val_loss: 28.4032 - val_accuracy: 0.4444
Epoch 2/2
1/1 [==============================] - 0s 24ms/step - loss: 27.1990 - accuracy: 0.8400 - val_loss: 28.2089 - val_accuracy: 0.4444
Restoring best model weights with validation accuracy: 0.4444444477558136


In [27]:
# fine-tune the network
history, features = model.extract_fine_tuned_features(x_train[:10], Y_train[:10, :], batch_size=5, epochs=2, validation_data=(x_test[:18], Y_test[:18,:]))

Epoch 1/2
2/2 [==============================] - 24s 12s/step - loss: 27.5368 - accuracy: 0.9000 - val_loss: 28.0939 - val_accuracy: 0.2778
Epoch 2/2
2/2 [==============================] - 16s 8s/step - loss: 26.1521 - accuracy: 0.9000 - val_loss: 27.0067 - val_accuracy: 0.3889
Restoring best model weights with validation accuracy: 0.31481482171350056


In [20]:
# save extractor
model.save_extractor("extractor.h5")

In [21]:
model.load_extractor("extractor.h5")

In [22]:
# save classification layer
model.save_classifier("classification.h5")

In [23]:
# load classicifation layer
model.load_classifier("classification.h5")